# Parse Tiny Tales data into a csv

In [1]:
import re
from pathlib import Path
import pandas as pd
import uuid
import tiktoken

In [2]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

## Define the directories and file paths

In [3]:
directory = Path("../text/TinyTales/")
out_directory = Path("../data")
glossary = directory/'glossary.txt'
stories = directory/'mahabharata.txt'
source = "http://100words.lauragibbs.net/"

## Preparing the glossary as table. 

In [4]:

glossary_text = glossary.read_text()
lines = glossary_text.split("\n")

glossary_list = list(map(lambda x: x.split(".", 1), lines))
df_glossary = pd.DataFrame(glossary_list, columns = ["entity", "description"])

df_glossary['entity'] = df_glossary.entity.str.strip()
df_glossary['description'] = df_glossary.description.str.strip()
df_glossary['source'] = source
df_glossary.head()

,entity,description,source
0,Abhimanyu,son of Arjuna and Subhadra; husband of Uttarai...,http://100words.lauragibbs.net/
1,Agni,fire-god.,http://100words.lauragibbs.net/
2,Airavata,elephant vehicle of the god Indra.,http://100words.lauragibbs.net/
3,Amba,daughter of the king of Kashi; later reincarna...,http://100words.lauragibbs.net/
4,Ambalika,daughter of the king of Kashi; wife of Vichitr...,http://100words.lauragibbs.net/


## Read the file with stories

In [5]:
stories_text = stories.read_text()
## Chapters follow this pattern: ** Chapter x: title **
## Subsection follows this pattern: ~ 1. Vyasa Seeks a Scribe ~

## Parse the file into dataframe 

Chapters follow this pattern:

`** Chapter x: title **`
 
Subsection follows this pattern: 

`~ 1. Vyasa Seeks a Scribe ~`

In [6]:

## Initial values
data = []
current_chapter = "No Chapter"
current_chapter_name = "No Chapter"
current_section_number = "No Section"
current_section_name = "No Section"
chapter_pattern = '(\*{2}.*\*{2}.*$)'
section_pattern = '(~{1}.*~{1}.*$)'

## Chapter Pattern: ** Chapter x: title ** 
chapters = re.split(chapter_pattern, stories_text, flags=re.MULTILINE)



### Split into sections

In [7]:

for chapter in chapters:
    if len(chapter.strip()) == 0:
        continue

    if re.match(chapter_pattern, chapter):
        chapter.replace('**', '')
        l = chapter.split(':')
        current_chapter = l[0].replace('**', '').replace('CHAPTER', '').strip()
        current_chapter_name = l[1].replace('**', '').strip()
        print("Chapter: ", current_chapter, current_chapter_name)
        continue
    
    ## Section Pattern: ~ 1. Vyasa Seeks a Scribe ~
    sections = re.split(section_pattern, chapter, flags=re.MULTILINE)
    for section in sections:
        if len(section.strip()) == 0:
            continue

        if re.match(section_pattern, section):
            l_s =  section.split('.', 1)
            current_section_number = l_s[0].replace('~', '').strip()
            current_section_name = l_s[1].replace('~', '').strip()
            print("Section: ", current_section_number, current_section_name)
        else:
            chapter_text = section
            chunk_id = uuid.uuid4().hex
            data = data + [[chapter_text, current_section_number, current_section_name, current_chapter, current_chapter_name, chunk_id]]


Chapter:  1 THE STORY BEGINS
Section:  1 Vyasa Seeks a Scribe
Section:  2 King Shantanu Gets Married
Section:  3 The Story of Mahabhisha
Section:  4 King Shantanu Confronts the Queen
Section:  5 The Story of the Vasus
Section:  6 Devavrata Departs and Returns
Section:  7 Shantanu Sees Satyavati
Section:  8 The Story of Satyavati
Section:  9 Devavrata Swears an Oath
Section:  10 King Shantanu Marries Satyavati
Section:  11 Bhishma Goes to the Swayamvara
Section:  12 Amba Pleads with Bhishma
Section:  13 Amba Goes to Parashurama
Section:  14 Amba Prays to the Gods
Section:  15 How the Sons of Satyavati Died
Section:  16 Vichitravirya Leaves Two Widows
Section:  17 Another Story of Satyavati
Section:  18 Satyavati Summons Vyasa
Section:  19 Three Sons are Born
Section:  20 The Story of Mandavya
Chapter:  2 PANDAVAS AND KAURAVAS
Section:  21 The Brothers Get Married
Section:  22 Pandu Goes Hunting
Section:  23 The Story of Kunti's Mantra
Section:  24 Kunti and Madri Have Sons
Section:  25 

### Write the dataframe to a file

In [8]:

df_stories = pd.DataFrame(
    data, 
    columns=[
        'text', 
        'section_number', 
        'title', 
        'chapter_number', 
        'chapter_name',
        'chunk_id',
        ]
)

## Convert the section number and chapter number to numeric
df_stories[['section_number', 'chapter_number']] = df_stories[['section_number', 'chapter_number']].astype('int')

## Add source of the material to the dataframe
df_stories['source'] = source

## Calculate number of tokens per row
df_stories['num_tokens'] = df_stories['text'].apply(num_tokens_from_string)

## Check the dataframe
df_stories.head()

,text,section_number,title,chapter_number,chapter_name,chunk_id,source,num_tokens
0,\nVyasa had composed a poem and needed a scrib...,1,Vyasa Seeks a Scribe,1,THE STORY BEGINS,acb483238ee7476f95e4d61ae181b9be,http://100words.lauragibbs.net/,156
1,\nKing Shantanu lived in a great palace in Has...,2,King Shantanu Gets Married,1,THE STORY BEGINS,f6a6888b2d9b4134aa260370993dd3dc,http://100words.lauragibbs.net/,133
2,"\nTo understand Shantanu's story, listen to th...",3,The Story of Mahabhisha,1,THE STORY BEGINS,612f8790b26e4995a8ac06e270f5817e,http://100words.lauragibbs.net/,154
3,\nKing Shantanu's queen drowned their first ch...,4,King Shantanu Confronts the Queen,1,THE STORY BEGINS,872c91a5f7224fbf87517daa94dec919,http://100words.lauragibbs.net/,132
4,"\nPrabhasa, chief of the Vasus, had coveted th...",5,The Story of the Vasus,1,THE STORY BEGINS,8999cc7405d2461ea0e41c31ca9c1b1f,http://100words.lauragibbs.net/,151


In [9]:
## Write to files
df_stories.to_csv(out_directory/'tiny_tales_summaries.csv', index=False, sep="|")
df_glossary.to_csv(out_directory/'tiny_tales_glossary.csv', index=False, sep="|")